In [27]:
import os
import numpy as np
from dfply import *
import pandas as pd
from scipy.stats import norm
from datetime import datetime
import matplotlib.pyplot as plt

In [62]:
자동차_원수_계약 = pd.read_excel('./data/자동차_표준손해율분포/자동차_원수_계약.xlsx', dtype={'FY': str, 'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str})
자동차_원수_보상 = pd.read_excel('./data/자동차_표준손해율분포/자동차_원수_보상.xlsx', dtype={'FY': str, 'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str})
자동차_출재_계약 = pd.read_excel('./data/자동차_표준손해율분포/자동차_출재_계약.xlsx', dtype={'FY': str, 'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str})
자동차_출재_보상 = pd.read_excel('./data/자동차_표준손해율분포/자동차_출재_보상.xlsx', dtype={'FY': str, 'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str})
자동차_상품군정보 = pd.read_excel('./data/자동차_표준손해율분포/자동차_상품군정보.xlsx', sheet_name='상품군분류', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str})
자동차_분류 = pd.read_excel('./data/자동차_충격시나리오/자동차_상품군정보.xlsx', sheet_name='KICS분류')

표준편차평균비율 = pd.read_excel('./data/자동차_표준손해율분포/자동차_보유리스크율_기초정보.xlsx', sheet_name='표준편차평균비율', dtype={'보종코드': str, '표준편차평균비율': float})
RMSE준비금비율 = pd.read_excel('./data/자동차_표준손해율분포/자동차_보유리스크율_기초정보.xlsx', sheet_name='RMSE준비금비율', dtype={'보종코드': str, 'RMSE준비금비율': float})
RMSE비중 = pd.read_excel('./data/자동차_표준손해율분포/자동차_보유리스크율_기초정보.xlsx', sheet_name='RMSE비중')

In [63]:
자동차_원수_계약_가공 = 자동차_원수_계약 >> left_join(자동차_상품군정보, on=['BSC_CVR_CD', 'PDGR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> left_join(자동차_분류, on=X.PDGR_NM)

자동차_원수_보상_가공 = 자동차_원수_보상 >> mask(X.AY != " ", ~X.AY.isna(), ~X.UY.isna()) \
    >> mutate(DY=X.FY.astype(int)-X.AY.astype(int)+1) \
    >> mask(X.DY >= 1) \
    >> mutate(LOSS=X.PYN_BNF+X.OST_BNF) \
    >> left_join(자동차_상품군정보, on=['BSC_CVR_CD', 'PDGR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> left_join(자동차_분류, on=X.PDGR_NM)

자동차_출재_계약_가공 = 자동차_출재_계약 >> left_join(자동차_상품군정보, on=['BSC_CVR_CD', 'PDGR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> left_join(자동차_분류, on=X.PDGR_NM)

자동차_출재_보상_가공 = 자동차_출재_보상 >> mask(X.AY != " ", ~X.AY.isna(), ~X.UY.isna()) \
    >> mutate(DY=X.FY.astype(int)-X.AY.astype(int)+1) \
    >> mask(X.DY >= 1) \
    >> mutate(RN_LOSS=X.RN_PYN_BNF+X.RN_OST_BNF) \
    >> left_join(자동차_상품군정보, on=['BSC_CVR_CD', 'PDGR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> left_join(자동차_분류, on=X.PDGR_NM)

RMSE비중_가공 = RMSE비중.copy()
RMSE비중_가공 = RMSE비중_가공.melt(id_vars=['구분'], value_vars=RMSE비중_가공.columns[1:]).dropna()
RMSE비중_가공.columns = ['UY', '보종명', 'RMSE비중']
RMSE비중_가공.insert(0, '보종코드', RMSE비중_가공.보종명.apply(lambda x: {'11.개인용자동차(인담보)': 'B001', '12.개인용자동차(물담보)': 'B002', '13.업무용자동차(인담보)': 'B003', '14.업무용자동차(물담보)': 'B004', '15.영업용자동차(인담보)': 'B005', '16.영업용자동차(물담보)': 'B006', '17.자동차기타': 'B007'}.get(x)).astype(str))
RMSE비중_가공['보종코드'] = RMSE비중_가공['보종코드'].astype(str)
RMSE비중_가공['UY'] = RMSE비중_가공['UY'].astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [104]:
기준년도 = 2018
result = []

산출기간 = 5
UY_전체 = np.arange(기준년도-산출기간+1, 기준년도+1).astype(str)
for UY in UY_전체:
    # 보종코드 추출
    출재계약_UY = 자동차_출재_계약_가공.query('UY == @UY')
    출재보상_UY = 자동차_출재_보상_가공.query('UY == @UY')
    보종코드목록 = 출재계약_UY.AT_PDGR_CD.drop_duplicates()

    for 보종코드 in 보종코드목록:

        출재계약_특약_UY_보종 = 출재계약_UY.query('AT_PDGR_CD == @보종코드')
        출재보상_특약_UY_보종 = 출재보상_UY.query('AT_PDGR_CD == @보종코드')

        # 출재 계약 (보험료)
        출재보험료_UY_국내_원수_보종 = 자동차_출재_계약_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['RN_WRTN_PRM'].sum()
        출재보험료_특약_UY_보종 = 출재계약_특약_UY_보종['RN_WRTN_PRM'].sum()
        출재보험료비중_특약_UY_보종 = 출재보험료_특약_UY_보종/출재보험료_UY_국내_원수_보종

        # 원수 계약 (보험료)
        원수보험료_UY_국내_원수_보종 = 자동차_원수_계약_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['WRTN_PRM'].sum()
        원수보험료_특약_UY_보종 = 원수보험료_UY_국내_원수_보종*출재보험료비중_특약_UY_보종

        # 출재 계약 (경과보험료)
        출재경과보험료_UY_국내_원수_보종 = 자동차_출재_계약_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['RN_ELP_PRM'].sum()
        출재경과보험료_특약_UY_보종 = 출재계약_특약_UY_보종['RN_ELP_PRM'].sum()
        출재경과보험료비중_특약_UY_보종 = 출재경과보험료_특약_UY_보종/출재경과보험료_UY_국내_원수_보종

        # 원수 계약 (경과보험료)
        원수경과보험료_UY_국내_원수_보종 = 자동차_원수_계약_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['ELP_PRM'].sum()
        원수경과보험료_특약_UY_보종 = 원수경과보험료_UY_국내_원수_보종*출재경과보험료비중_특약_UY_보종

        # 출재 보상 (보험금)
        출재보험금_UY_국내_원수_보종 = 자동차_출재_보상_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['RN_PYN_BNF'].sum()
        출재보험금_특약_UY_보종 = 출재보상_특약_UY_보종['RN_PYN_BNF'].sum()
        출재보험금비중_특약_UY_보종 = 출재보험금_특약_UY_보종/출재보험금_UY_국내_원수_보종

        # 원수 보상 (보험금)
        원수보험금_UY_국내_원수_보종 = 자동차_원수_보상_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['PYN_BNF'].sum()
        원수보험금_특약_UY_보종 = 원수보험금_UY_국내_원수_보종*출재보험금비중_특약_UY_보종

        # 출재 보상 (준비금)
        출재준비금_UY_국내_원수_보종 = 자동차_출재_보상_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['RN_OST_BNF'].sum()
        출재준비금_특약_UY_보종 = 출재보상_특약_UY_보종['RN_OST_BNF'].sum()
        출재준비금비중_특약_UY_보종 = 0 if 출재준비금_UY_국내_원수_보종 == 0 else 출재준비금_특약_UY_보종/출재준비금_UY_국내_원수_보종

        # 원수 보상 (준비금)
        원수준비금_UY_국내_원수_보종 = 자동차_원수_보상_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['OST_BNF'].sum()
        원수준비금_특약_UY_보종 = 원수준비금_UY_국내_원수_보종*출재준비금비중_특약_UY_보종

        # 출재 보상 (손해액)
        출재손해액_특약_UY_보종 = 출재보험금_특약_UY_보종 + 출재준비금_특약_UY_보종

        # 원수 보상 (손해액)
        원수손해액_특약_UY_보종 = 원수보험금_특약_UY_보종 + 원수준비금_특약_UY_보종
        특약명 = '자동차비례재보험특약'
        컬럼 = ['특약명', 'UY', '보종코드', '원수보험료', '출재보험료', '원수경과보험료', '출재경과보험료', '원수보험금', '출재보험금', '원수준비금', '출재준비금', '원수손해액', '출재손해액']
        데이터 = [특약명, UY, 보종코드, 원수보험료_특약_UY_보종, 출재보험료_특약_UY_보종, 원수경과보험료_특약_UY_보종, 출재경과보험료_특약_UY_보종, 원수보험금_특약_UY_보종, 출재보험금_특약_UY_보종, 원수준비금_특약_UY_보종, 출재준비금_특약_UY_보종, 원수손해액_특약_UY_보종, 출재손해액_특약_UY_보종]
        df = pd.DataFrame(데이터, 컬럼).T
        result.append(df)
df = pd.concat(result)

In [105]:
# 수수료 식
def auto_commission(UY, R):
    if UY == '2018':
        if R <= 0.75:
            C = 0.24
        elif R <= 0.85:
            C = 0.9945 - R
        elif R <= 0.95:
            C = 0.9950 - R
        elif R <= 0.9955:
            C = 0.9955 - R
        else:
            C = 0
    elif UY == '2017':
        if R <= 0.75:
            C = 0.24
        elif R <= 0.85:
            C = 0.9945 - R
        elif R <= 0.95:
            C = 0.9950 - R
        elif R <= 1.05:
            C = 0.9955 - R
        elif R <= 1.15:
            C = 0.9960 - R
        elif R <= 1.25:
            C = 1 - R
        else:
            C = (1-R)*0.6 + 0.1
    elif UY == '2016':
        if R <= 0.75:
            C = 0.24
        elif R <= 0.90:
            C = 0.992 - R
        elif R <= 1.10:
            C = 0.995 - R
        elif R <= 1.25:
            C = 0.998 - R
        elif R <= 1.45:
            C = (1-R)*0.9
        else:
            C = (1-R)*0.6
    elif UY == '2015':
        if R <= 0.80:
            C = 0.18
        elif R <= 0.90:
            C = 0.99 - R
        elif R <= 1.05:
            C = 0.995 - R
        elif R <= 1.30:
            C = 1.00 - R
        else:
            C = 0
    elif UY == '2014':
        if R <= 0.80:
            C = 0.18
        elif R <= 0.90:
            C = 0.99 - R
        elif R <= 1.05:
            C = 0.995 - R
        elif R <= 1.30:
            C = 1.00 - R
        else:
            C = 0
    else:
        raise Exception('유효하지 않은 UY입니다.')
    return C

In [106]:
df['보종코드'] = df.보종코드.astype(str)
df['원수손해율'] = df['원수손해액']/df['원수경과보험료']
df2 = df.groupby(['특약명', 'UY'])[['원수보험료', '출재보험료', '원수준비금', '출재준비금', '원수보험금', '출재보험금', '원수손해액', '출재손해액']].sum().reset_index()
df2.columns = ['특약명', 'UY', '원수보험료_특약', '출재보험료_특약', '원수준비금_특약', '출재준비금_특약', '원수보험금_특약', '출재보험금_특약', '원수손해액_특약', '출재손해액_특약']
df3 = df.merge(표준편차평균비율, on='보종코드', how='left').drop('보종명', axis=1)
df3['원수손해율'] = df3['원수손해율'].astype(float)
df3['원수손해율변동성'] = df3['원수손해율']*df3['표준편차평균비율']
df3['MU'] = df3.query('원수손해율 > 0').eval('MU = 2*log(원수손해율)-log(원수손해율**2+원수손해율변동성**2)/2')['MU']
df3['MU'] = df3['MU'].fillna(0)
df3['SIGMA'] = df3.eval('SIGMA = sqrt(log(원수손해율**2+원수손해율변동성**2) - 2*log(원수손해율))')['SIGMA']
df3['SIGMA'] = df3['SIGMA'].fillna(0)
df3 = df3.merge(RMSE비중_가공, on=['보종코드', 'UY'], how='left').drop('보종명', axis=1)
df3['원수준비금'] = df3['원수준비금'].astype(float)
temp = df3.groupby(['특약명', '보종코드'])[['원수준비금']].sum().reset_index()
temp.columns = ['특약명', '보종코드', '보종별원수준비금']
df3 = df3.merge(RMSE준비금비율, on='보종코드', how='left').drop('보종명', axis=1)
df3 = df3.merge(temp, on=['특약명', '보종코드'], how='left')
df3.eval('원수준비금RMSE = 보종별원수준비금*RMSE준비금비율*RMSE비중', inplace=True)
df3 = df3.drop(['RMSE비중', 'RMSE준비금비율', '보종별원수준비금'], axis=1)
df3 = df3.merge(df2, on=['특약명', 'UY'], how='left')
df3.head()

,특약명,UY,보종코드,원수보험료,출재보험료,원수경과보험료,출재경과보험료,원수보험금,출재보험금,원수준비금,...,SIGMA,원수준비금RMSE,원수보험료_특약,출재보험료_특약,원수준비금_특약,출재준비금_특약,원수보험금_특약,출재보험금_특약,원수손해액_특약,출재손해액_특약
0,자동차비례재보험특약,2014,B005,7.96403e+10,219280710,7.96403e+10,219280710,6.47762e+10,200635,0.000000e+00,...,0.078053,2.544572e+07,2.055853e+11,3833976438,0.000000e+00,0,1.752630e+11,35713508,1.752630e+11,35713508
1,자동차비례재보험특약,2014,B006,1.03996e+10,17745234,1.03996e+10,17745234,8.51857e+09,1423242,0.000000e+00,...,0.068815,0.000000e+00,2.055853e+11,3833976438,0.000000e+00,0,1.752630e+11,35713508,1.752630e+11,35713508
2,자동차비례재보험특약,2014,B007,1.15545e+11,3596950494,1.15545e+11,3596950494,1.01968e+11,34089631,0.000000e+00,...,0.354511,3.909660e+07,2.055853e+11,3833976438,0.000000e+00,0,1.752630e+11,35713508,1.752630e+11,35713508
3,자동차비례재보험특약,2015,B005,8.87975e+10,4919726586,8.87975e+10,4919726586,6.7734e+10,358195102,3.558952e+09,...,0.078053,4.287266e+07,1.827621e+12,11911166777,1.834075e+10,54628614,1.418167e+12,690155275,1.436507e+12,744783889
4,자동차비례재보험특약,2015,B001,5.42719e+11,-3081297,5.42719e+11,-3081297,4.44096e+11,10816168,1.395247e+10,...,0.065965,4.804882e+08,1.827621e+12,11911166777,1.834075e+10,54628614,1.418167e+12,690155275,1.436507e+12,744783889


In [107]:
# 보험가격 보유리스크율
result = []
for _, (특약명, UY, 보종코드) in df[['특약명', 'UY', '보종코드']].iterrows():
    data = df3.query('특약명 == @특약명').query('UY == @UY').query('보종코드 == @보종코드').iloc[0]

    mu = data['MU']
    sigma = data['SIGMA']
    원수보험료 = data['원수보험료']
    원수손해액 = data['원수손해액']
    출재보험료 = data['출재보험료']
    출재보험료_특약 = data['출재보험료_특약']
    출재손해액 = data['출재손해액']
    원수손해액_특약 = data['원수손해액_특약']
    출재손해액_특약 = data['출재손해액_특약']
#     auto_commission(UY, 출재손해율_특약_난수)

    np.random.seed(20190814)
    난수 = np.random.random(10000)
    명목보유율 = (원수보험료-출재보험료)/원수보험료
    원수손해율_난수 = np.exp(norm(loc=mu, scale=sigma).ppf(난수))
    원수손해액_난수 = 원수손해율_난수*원수보험료
    원수손해액_특약_난수 = 원수손해액_특약-원수손해액+원수손해액_난수
    원수기대손해액 = 원수손해액_특약_난수.mean()
    원수리스크 = np.fmax(원수손해액_특약_난수-원수기대손해액, 0)
    원수기대손실액 = 원수리스크.mean()
    보유손해액_특약_난수 = (원수손해액_특약-출재손해액_특약)-(원수손해액-출재손해액)+원수손해액_난수*명목보유율
    출재손해율_특약_난수 = (원수손해액_특약_난수 - 보유손해액_특약_난수)/출재보험료_특약
    수수료정산액 = 출재보험료_특약*np.array(list(map(lambda x: auto_commission(UY, x), 출재손해율_특약_난수)))
    보유현금흐름 = 보유손해액_특약_난수 - 수수료정산액
    보유기대손해액 = 보유현금흐름.mean()
    보유리스크 = np.fmax(보유현금흐름-보유기대손해액, 0)
    보유기대손실액 = 보유리스크.mean()
    보유리스크율 = 보유기대손실액/원수기대손실액
    보유율차이 = 보유리스크율 - 명목보유율

    컬럼명 = ['특약명', 'UY', '보종코드', '원수기대손해액', '원수기대손실액', '보유기대손해액', '보유기대손실액', '보유리스크율', '명목보유율', '보유율차이']
    dg = pd.DataFrame([특약명, UY, 보종코드, 원수기대손해액, 원수기대손실액, 보유기대손해액, 보유기대손실액, 보유리스크율, 명목보유율, 보유율차이], 컬럼명).T
    result.append(dg)
dg = pd.concat(result)
dg

,특약명,UY,보종코드,원수기대손해액,원수기대손실액,보유기대손해액,보유기대손실액,보유리스크율,명목보유율,보유율차이
0,자동차비례재보험특약,2014,B005,1.75221e+11,1.96488e+09,1.74317e+11,1.95947e+09,0.997247,0.997247,6.88338e-15
0,자동차비례재보험특약,2014,B006,1.75258e+11,2.27878e+08,1.74519e+11,2.27489e+08,0.998294,0.998294,2.37588e-14
0,자동차비례재보험특약,2014,B007,1.74706e+11,1.38621e+10,1.71168e+11,1.36057e+10,0.981499,0.96887,0.0126291
0,자동차비례재보험특약,2015,B005,1.43646e+12,2.16256e+09,1.43003e+12,2.04274e+09,0.944596,0.944596,4.55191e-15
0,자동차비례재보험특약,2015,B001,1.43627e+12,1.17466e+10,1.43339e+12,1.17467e+10,1.00001,1.00001,-1.31006e-14
0,자동차비례재보험특약,2015,B007,1.43592e+12,1.47057e+10,1.42808e+12,1.40091e+10,0.952632,0.950691,0.00194036
0,자동차비례재보험특약,2015,B002,1.43605e+12,2.19642e+10,1.43318e+12,2.19642e+10,1,1,1.33227e-15
0,자동차비례재보험특약,2015,B006,1.4365e+12,4.96773e+08,1.43332e+12,4.89083e+08,0.98452,0.98452,1.85518e-13
0,자동차비례재보험특약,2016,B001,1.42771e+12,1.29819e+10,1.41169e+12,1.29143e+10,0.994797,0.993268,0.00152916
0,자동차비례재보험특약,2016,B002,1.42749e+12,2.31174e+10,1.41131e+12,2.31174e+10,0.999997,0.993288,0.00670894


In [108]:
# 준비금 보유리스크율
result = []
for _, (특약명, UY, 보종코드) in df[['특약명', 'UY', '보종코드']].iterrows():
    data = df3.query('특약명 == @특약명').query('UY == @UY').query('보종코드 == @보종코드').iloc[0]

    원수준비금 = data['원수준비금']
    원수준비금_특약 = data['원수준비금_특약']
    출재준비금 = data['출재준비금']
    출재준비금_특약 = data['출재준비금_특약']
    출재보험료_특약 = data['출재보험료_특약']
    출재보험금_특약 = data['출재보험금_특약']
    출재손해액_특약 = data['출재손해액_특약']   # 출재손해액 정의 다시하기(손조사비 제외하는게 맞는 듯)
    원수준비금RMSE = data['원수준비금RMSE']

    명목보유율 = (원수준비금-출재준비금)/원수준비금 if 원수준비금 != 0 else 0
    출재손해율_특약 = 출재손해액_특약/출재보험료_특약
    
    수수료정산액 = 출재보험료_특약*auto_commission(UY, 출재손해율_특약)

    np.random.seed(20190814)
    난수 = np.random.random(10000)
    원수준비금_난수 = norm(loc=원수준비금, scale=원수준비금RMSE).ppf(난수)
    원수준비금_특약_난수 = 원수준비금_특약 + (원수준비금_난수 - 원수준비금)
    원수기대준비금 = 원수준비금_특약_난수.mean()
    원수리스크 = np.fmax(원수준비금_특약_난수-원수기대준비금, 0)
    원수기대준비금부족액 = 원수리스크.mean()
    보유준비금_특약_난수 = (원수준비금_특약 - 출재준비금_특약) - (원수준비금 - 출재준비금) + 원수준비금_난수*명목보유율
    출재손해율_특약_난수 = (원수준비금_특약_난수-보유준비금_특약_난수+출재보험금_특약)/출재보험료_특약
    수수료차액 = 출재보험료_특약*np.array(list(map(lambda x: auto_commission(UY, x), 출재손해율_특약_난수))) - 수수료정산액
    
    보유현금흐름 = 보유준비금_특약_난수-수수료차액
    보유기대준비금 = 보유현금흐름.mean()
    보유리스크 = np.fmax(보유현금흐름-보유기대준비금, 0)
    
    보유기대준비금부족액 = 보유리스크.mean()
    보유리스크율 = 보유기대준비금부족액/원수기대준비금부족액
    보유율차이 = 보유리스크율 - 명목보유율

    컬럼명 = ['특약명', 'UY', '보종코드', '원수기대준비금', '원수기대준비금부족액', '보유기대준비금', '보유기대준비금부족액', '보유리스크율', '명목보유율', '보유율차이']
    dh = pd.DataFrame([특약명, UY, 보종코드, 원수기대준비금, 원수기대준비금부족액, 보유기대준비금, 보유기대준비금부족액, 보유리스크율, 명목보유율, 보유율차이], 컬럼명).T
    result.append(dh)
dh = pd.concat(result)
dh

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1983: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1984: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


,특약명,UY,보종코드,원수기대준비금,원수기대준비금부족액,보유기대준비금,보유기대준비금부족액,보유리스크율,명목보유율,보유율차이
0,자동차비례재보험특약,2014,B005,-164181,9.9086e+06,0,0,0,0,0
0,자동차비례재보험특약,2014,B006,NaN,0,NaN,0,NaN,0,NaN
0,자동차비례재보험특약,2014,B007,-252260,1.52243e+07,0,0,0,0,0
0,자동차비례재보험특약,2015,B005,1.83405e+10,1.66947e+07,1.82858e+10,1.64595e+07,0.98591,0.98591,-7.26086e-14
0,자동차비례재보험특약,2015,B001,1.83376e+10,1.87103e+08,1.8283e+10,1.87061e+08,0.999774,0.999774,3.66374e-15
0,자동차비례재보험특약,2015,B007,1.83405e+10,1.78029e+07,1.82858e+10,1.77228e+07,0.9955,0.9955,-3.09752e-14
0,자동차비례재보험특약,2015,B002,1.83403e+10,2.77262e+07,1.82857e+10,2.77095e+07,0.999395,0.999395,3.85247e-14
0,자동차비례재보험특약,2015,B006,NaN,0,NaN,0,NaN,0,NaN
0,자동차비례재보험특약,2016,B001,2.97085e+10,2.33987e+08,2.90987e+10,2.33987e+08,1,0.989341,0.0106587
0,자동차비례재보험특약,2016,B002,2.97115e+10,4.91121e+07,2.91017e+10,4.91121e+07,1,0.9702,0.0297998


In [109]:
if not any([s=='result' for s in os.listdir('.')]):
    os.mkdir('result')
now = datetime.now().strftime('%Y%m%d%H%M%S')
ver = 'v1.0.0'

with pd.ExcelWriter('./result/자동차_표준손해율분포_{}_{}.xlsx'.format(ver, now)) as writer:
    df3.to_excel(writer, '자동차_연동수수료특약_UY별_실적집계', index=False)
    dg.to_excel(writer, '자동차_연동수수료특약_표준분포법_보험가격', index=False)
    dh.to_excel(writer, '자동차_연동수수료특약_표준분포법_준비금', index=False)